# Data Explorer Notebook

## Data Description

### [FinnGen](https://finngen.gitbook.io/documentation/)

- The FinnGen research project is an expedition to the frontier of genomics and medicine, with significant discoveries potentially arising from any one of Finland’s 500,000 biomedical pioneers.
- The project brings together a nation-wide network of Finnish biobanks, with every Finn able to participate in the study by giving biobank consent.
- As of the last update, there were 589,000 samples available, with a goal to reach 520,000 by 2023. The latest data freeze included combined genotype and health registry data from 473,681 individuals.
- The study utilizes samples collected by a nationwide network of Finnish biobanks and combines genome information with digital health care data from national health registries.
- There's a need for samples from all over Finland as solutions in the field of personalized healthcare can be found only by looking at large populations. Every Finn can be a part of the FinnGen study by giving a biobank consent.
- The genome data produced during the project is owned by the Finnish biobanks and remains available for research purposes. The medical breakthroughs that arise from the project are expected to benefit health care systems and patients globally.
- The FinnGen research project is collaborative, involving all the same actors as drug development, with the aim to speed up the emergence of new innovations.
- The project's data freeze 9 results and summary statistics are now available, consisting of over 377,200 individuals, almost 20.2 M variants, and 2,272 disease endpoints. Results can be browsed online using the FinnGen web browser, and the summary statistics downloaded.
- The University of Helsinki is the organization responsible for the study, and the nationwide network of Finnish biobanks is participating in the study, thus covering the whole of Finland. The Helsinki Biobank coordinates the sample collection.
- For more information, the project can be contacted at finngen-info@helsinki.fi.

## Purpose of Notebook

### Final Ouput

The DataFrame has the following structure and columns:

- Total Rows: 20,170,006
- Columns: 16

Column Names and Descriptions:

1. #chrom: Chromosome number based on the GRCh38 build (1-23).
2. pos: Position of the variant in base pairs on the GRCh38 build.
3. ref: Reference allele for the variant.
4. alt: Alternative allele (effect allele) for the variant.
5. rsids: Identifier for the variant.
6. nearest_genes: Nearest gene(s) to the variant, separated by commas.
7. pval: p-value associated with the variant. 
8. mlogp: Negative logarithm (base 10) of the p-value (log10(p-value)).
9. beta: Effect size of the variant on a log odds ratio (OR) scale, estimated for the alternative allele.
10. sebeta: Standard error of the effect size estimated for the variant.
11. af_alt: Alternative (effect) allele frequency.
12. af_alt_cases: Alternative (effect) allele frequency among cases.
13. af_alt_controls: Alternative (effect) allele frequency among controls.
14. causal: Defines whether the SNP is causal or not with the trait (1 = causal, 0 = not).
15. LD: Defines whether the SNP is in linkage disequilibrium (LD) with the lead variant (r^2 > 0.8).
16. lead: rsid of the lead variant SNP.
17. trait: phenotype


The memory usage of the DataFrame is approximately 2.3 GB.

## Import data

In [1]:
import pandas as pd

In [2]:
finemap = pd.read_csv('C:/Users/falty/Desktop/geometric-omics/FinnGenn/data/finemapping_full_finngen_R9_T2D.SUSIE.snp.tsv', low_memory=False, sep='\t')
causal = pd.read_csv('C:/Users/falty/Desktop/geometric-omics/FinnGenn/data/41467_2021_25514_MOESM8_ESM.csv', low_memory=False)
gwas = pd.read_csv('C:/Users/falty/Desktop/geometric-omics/FinnGenn/data/summary_stats_finngen_R9_T2D.tsv', low_memory=False, sep='\t')

In [3]:
print(len(finemap))
print(len(causal))
print(len(gwas))

3057847
43
20170006


In [4]:
# Assuming gwas is your DataFrame
null_values = gwas.isnull().sum()

# Printing the null values count for each column
print(null_values)

#chrom                   0
pos                      0
ref                      0
alt                      0
rsids              1366396
nearest_genes       727855
pval                     0
mlogp                    0
beta                     0
sebeta                   0
af_alt                   0
af_alt_cases             0
af_alt_controls          0
dtype: int64


In [5]:
# Extract number from 'chromosome' and replace 'X' with '23'
finemap['chromosome'] = finemap['chromosome'].str.extract('(\d+|X)', expand=False).replace('X', '23')

# Convert 'chromosome' column to 'int64'
finemap['chromosome'] = finemap['chromosome'].astype('int64')

## Create `causal` col in `gwas`

In [6]:
gwas['causal'] = gwas['rsids'].isin(causal['SNP']).astype(int)

## Create `LD` col in `gwas`

In [8]:
gwas['#chrom']

0            1
1            1
2            1
3            1
4            1
            ..
20170001    23
20170002    23
20170003    23
20170004    23
20170005    23
Name: #chrom, Length: 20170006, dtype: int64

In [9]:
# Filter finemap by 'lead_r2' between 0.8 (inclusive) and 1 (exclusive)
filtered_finemap = finemap[(finemap['lead_r2'] >= 0.8) & (finemap['lead_r2'] < 1)].copy()

# Extract the position from 'v' column in filtered_finemap
filtered_finemap.loc[:, 'position'] = filtered_finemap['v'].str.split(':').str.get(1).astype(int)

# Create multi-index for fast selection and sort the index
filtered_finemap.set_index(['chromosome', 'position'], inplace=True)
filtered_finemap.sort_index(inplace=True)

# Create a set of tuples from the index of filtered_finemap
filtered_finemap_tuples = set(filtered_finemap.index)

# Map '#chrom' and 'pos' tuples from gwas to their existence in filtered_finemap_tuples
gwas['LD'] = list(map(int, gwas[['#chrom', 'pos']].apply(tuple, axis=1).isin(filtered_finemap_tuples)))

ld_sum = gwas['LD'].sum()

print(len(filtered_finemap))
print(ld_sum)

4085
4143


## Create `lead` col in `gwas`

In [10]:
import numpy as np

# Step 1: Filter the lead variants from the finemap dataset
lead_variants = finemap[finemap['lead_r2'] == 1].copy()

# Extract the position from 'v' column in lead_variants
lead_variants['position'] = lead_variants['v'].str.split(':').str.get(1).astype(int)

# Create multi-index for fast selection and sort the index
lead_variants.set_index(['chromosome', 'position'], inplace=True)
lead_variants.sort_index(inplace=True)

# Step 2: Initialize the 'lead' column in the gwas dataset
gwas['lead'] = ''

# Step 3: Loop over the gwas dataset
for idx, row in gwas[gwas['LD'] == 1].iterrows():
    # Get the same chromosome variants from the lead_variants
    same_chr_variants = lead_variants.loc[row['#chrom']]
    
    # If there are no lead variants on the same chromosome, skip this iteration
    if same_chr_variants.empty:
        continue

    # Calculate the absolute difference in position and get the rsid of the variant with the closest position
    position_difference = np.abs(pd.Series(same_chr_variants.index) - row['pos'])
    closest_position = position_difference.idxmin()
    lead_rsid = same_chr_variants.iloc[closest_position]['rsid']
    
    # Add the rsid to the 'lead' column in the gwas dataset
    gwas.at[idx, 'lead'] = lead_rsid

gwas['lead'] = gwas['lead'].replace('', 'NA')
lead_counts = gwas['lead'].value_counts().loc[lambda x: x.index != 'NA'].sum()
print(lead_counts)

265
Index(['trait', 'region', 'v', 'rsid', 'allele1', 'allele2', 'maf', 'beta',
       'se', 'p', 'mean', 'sd', 'prob', 'cs', 'cs_specific_prob', 'low_purity',
       'lead_r2', 'mean_99', 'sd_99', 'prob_99', 'cs_99',
       'cs_specific_prob_99', 'low_purity_99', 'lead_r2_99', 'alpha1',
       'alpha2', 'alpha3', 'alpha4', 'alpha5', 'alpha6', 'alpha7', 'alpha8',
       'alpha9', 'alpha10', 'mean1', 'mean2', 'mean3', 'mean4', 'mean5',
       'mean6', 'mean7', 'mean8', 'mean9', 'mean10', 'sd1', 'sd2', 'sd3',
       'sd4', 'sd5', 'sd6', 'sd7', 'sd8', 'sd9', 'sd10', 'lbf_variable1',
       'lbf_variable2', 'lbf_variable3', 'lbf_variable4', 'lbf_variable5',
       'lbf_variable6', 'lbf_variable7', 'lbf_variable8', 'lbf_variable9',
       'lbf_variable10'],
      dtype='object')
4143


In [13]:
unique_trait = finemap['trait'].unique()
trait_string = unique_trait[0]
gwas['trait'] = trait_string

In [14]:
gwas.columns

Index(['#chrom', 'pos', 'ref', 'alt', 'rsids', 'nearest_genes', 'pval',
       'mlogp', 'beta', 'sebeta', 'af_alt', 'af_alt_cases', 'af_alt_controls',
       'causal', 'LD', 'lead', 'trait'],
      dtype='object')

In [15]:
gwas.to_csv('gwas-causal.csv', index=False)